In [1]:
import pandas as pd
import numpy as np
import wget
import os

In [40]:
county_data = pd.read_csv("us-counties.csv")

In [61]:
nulls = county_data[county_data.isna().any(axis=1)]

In [73]:
nulls.to_csv("nulls.csv")
# All rows where FIPS is null. This consists of (a) New York City (all reported as one unit), (b) various Unknowns. 
# See NYT documentation on geographical exceptions.

In [94]:
# Download daily data files -- did this on 5/20. Saved as .txt files in /data

for m in range(4):
    for d in range(31):
        month = str(m+2).zfill(2)
        day = str(d+1).zfill(2)
        date = "2020"+month+day
        url = "https://s3-us-west-1.amazonaws.com//files.airnowtech.org/airnow/2020/"+date+"/daily_data_v2.dat"
        try:
            wget.download(url, "./data/"+date+".txt")
            print('Downloading', date)
        except:
            print('No file for date', date)

No file for date 20200230
No file for date 20200231
No file for date 20200431
No file for date 20200521
No file for date 20200522
No file for date 20200523
No file for date 20200524
No file for date 20200525
No file for date 20200526
No file for date 20200527
No file for date 20200528
No file for date 20200529
No file for date 20200530
No file for date 20200531


In [138]:
# make DFs from .txt files
# date_dfs is a dictionary of DFs where keys are dates in MMDD format

# DF column values: "aqsid" = site ID, first 5 digits are FIPS code; 
# "period" = averaging period (either peak 1-hr, peak 8-hr average, or 24-hr average);
# "aqi_value" = 8-hr or 24-hr AQI value, -999 = peak 1-hr ozone or peak 24-hr SO2; "aqi_category" = 0=good etc.;
# "aqsid_full" = AQS site ID preceded by 3-digit country code

date_dfs = {}
allfiles = os.listdir('data')[1:]
for filename in allfiles:
    date = filename[4:8]
    df = pd.read_csv('data/'+filename, sep="|", header=None)
    df.columns = ["date","aqsid","sitename","parameter","units","value","period","source","aqi_value","aqi_category","latitude","longitude","aqsid_full"]
    df['fips'] = df['aqsid'].str.slice(stop=5)  # add FIPS column based on first 5 digits of AQSID
    date_dfs[date] = df

In [170]:
date_dfs['0520'] = None   # dropped today's table because it contained data from 5/18 for some reason

In [114]:
# table of FIPS codes to county/state

fips = pd.read_html('https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697')[0]

In [148]:
fips = fips.drop(3232) # last row was gibberish

In [154]:
fips = fips.rename(columns={"Name":"County"})

In [175]:
fips = fips.set_index('FIPS')

In [177]:
date_dfs['0519']

,date,aqsid,sitename,parameter,units,value,period,source,aqi_value,aqi_category,latitude,longitude,aqsid_full,fips
0,05/19/20,000010102,St. John's,OZONE-1HR,PPB,31.0,1,Newfoundland & Labrador DEC,-999,-999,47.652800,-52.81670,124000010102,00001
1,05/19/20,000010102,St. John's,OZONE-8HR,PPB,29.0,8,Newfoundland & Labrador DEC,27,0,47.652800,-52.81670,124000010102,00001
2,05/19/20,000010401,Mount Pearl,OZONE-1HR,PPB,30.0,1,Newfoundland & Labrador DEC,-999,-999,47.505000,-52.79470,124000010401,00001
3,05/19/20,000010401,Mount Pearl,OZONE-8HR,PPB,28.0,8,Newfoundland & Labrador DEC,26,0,47.505000,-52.79470,124000010401,00001
4,05/19/20,000010401,Mount Pearl,PM2.5-24hr,UG/M3,4.7,24,Newfoundland & Labrador DEC,20,0,47.505000,-52.79470,124000010401,00001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,05/19/20,TT9209004,Miami,PM2.5-24hr,UG/M3,2.2,24,Quapaw Nation,10,0,36.922222,-94.83889,840TT9209004,TT920
4273,05/19/20,UB1010001,Ulaanbaatar,PM2.5-24hr,UG/M3,14.3,24,U.S. Department of State Mongolia - Ulaanbaatar,56,1,47.928385,106.92947,496UB1010001,UB101
4274,05/19/20,UZB010001,Tashkent,OZONE-1HR,PPB,51.0,1,U.S. Department of State Uzbekistan - Tashkent,-999,-999,41.367200,69.27250,860UZB010001,UZB01
4275,05/19/20,UZB010001,Tashkent,OZONE-8HR,PPB,46.0,8,U.S. Department of State Uzbekistan - Tashkent,43,0,41.367200,69.27250,860UZB010001,UZB01


In [ ]:
# drop non-US data
# drop unneeded columns
# make SQL database with one massive table?